## Data Collection

This notebook was used to analyze the errors made by the best features

In [1]:
import pandas as pd
from collections import defaultdict
from pprint import pprint
pd.options.display.max_rows = 1500

In [2]:
conf = pd.read_csv('../../code/data/data.csv')[['id','most_frequent_percentage']] # full data
conf.set_index('id')
df_cols = ['id','caption','sentence','object_a','object_b','predicted','gold', 'most_frequent_percentage']
paths = pd.read_csv('../../code/data/middle_paths_unrestricted_16.csv') # path embeddings
paths['sentence'] = paths.apply(lambda row: row['sentence'][2:-2],axis=1)
data = pd.read_csv('../../all_data_files/experiments/results_3_classes/missclassified_binary_False.csv') # missclassified
pd.options.display.max_colwidth = 100000

In [3]:
paths[paths.paths.str.len() < 20].paths.value_counts()

[['NOPATH']]    399
Name: paths, dtype: int64

In [4]:
# all errors made by sentence embeddings
se = data[data.caption == "InferSent"]
se = se.join(conf,rsuffix="_r")[df_cols]

In [5]:
# all errors for the lexnet feature
lex = data[data.caption == 'middle_paths_unrestricted_16']
#lex.set_index('id')
lex = lex.join(conf,rsuffix="_r")[df_cols]
print(len(lex))

1091


In [6]:
# all errors exclusivly made by se
se_unique = se[~se.sentence.isin(lex.sentence.values.tolist())].copy()
se_unique['origin'] = 'se (unique)'

In [7]:
p = paths[['sentence','paths','id']].copy()

# errors exclusivly made by lexnet
lex_unique = lex[~lex.sentence.isin(se.sentence.values.tolist())].copy()
lex_unique['origin'] = 'lex (unique)'
lex_unique = lex_unique.join(p,rsuffix='r')
lex_unique = lex_unique.drop_duplicates(subset=['sentence'])
lex_unique

,id,caption,sentence,object_a,object_b,predicted,gold,most_frequent_percentage,origin,sentencer,paths,idr
1410,134,middle_paths_unrestricted_16,From (1) and (2) follows this question: Is it possible at all to write a web framework in Ruby that's faster than PHP?,Ruby,PHP,NONE,BETTER,1.000000,lex (unique),"Like michigan, pennsylvania is within a point of moving into the safer', 'Obama lean category.",[['X/PROPN/ROOT/^ Y/PROPN/conj/<']],K1630595994
1411,830,middle_paths_unrestricted_16,"Even without the heavy functional stuff, just as a basic OO language, Scala is far better than C++/Java/C#.",Scala,Java,NONE,BETTER,1.000000,lex (unique),"Obama did better in North carolina than Hillary had done in', 'pennsylvania and Indiana was considered too close to call until well', 'past midnight.","[['X/PROPN/ROOT/^ Y/PROPN/pobj/<'], ['X/PROPN/ROOT/^ do/VERB/advcl/V Y/PROPN/pobj/<']]",L1858700103
1412,378,middle_paths_unrestricted_16,"Again, this is a decent beer but its not my cup of tea.",beer,tea,WORSE,NONE,1.000000,lex (unique),"Why can't Ford puts better handling into the FWD Fusion if Honda can"", 'do it 10 years ago?",[['X/PROPN/nsubj/> put/VERB/ROOT/^ if/ADP/prep/< Y/PROPN/pobj/<']],T3173142468
1413,1618,middle_paths_unrestricted_16,"The sausage is decent too, three kinds sold by the sandwich ($7.50) or by the pound ($15.75).",sausage,sandwich,BETTER,NONE,1.000000,lex (unique),"georgia looked much better against missouri, particularly in the', 'second half than it did at any point against Buffalo and might be', 'putting things together now.","[['X/PROPN/nsubj/> look/VERB/ROOT/^ better/ADV/acomp/< against/ADP/prep/<', 'Y/PROPN/pobj/<']]",F804953248
1416,504,middle_paths_unrestricted_16,Java apps on Mac are slower and bulkier than their native Objective-C counterparts.,Java,Objective-C,BETTER,WORSE,1.000000,lex (unique),"With the wii remote (and later the shameless copy, the playstation', 'Move), faster precision was never this simple!","[['X/PROPN/ROOT/^ Y/PROPN/dobj/<'], ['X/PROPN/ROOT/^ remote/ADJ/nmod/V Y/PROPN/dobj/<'], ['X/PROPN/ROOT/^ the/DET/det/V Y/PROPN/dobj/<']]",T3203049748
1417,13,middle_paths_unrestricted_16,"As good as michigan has been defensively this season, virginia has been even better.",michigan,virginia,NONE,WORSE,0.800000,lex (unique),"We say 'Java is the gateway to hell.' Lisp is just a better"", 'alternative to the intelligent Internet: It runs on 14 different', ""platforms and doesn't need any type of virtual machine because it"", 'compiles directly to native machine instructions on all popular', ""microprocessor architectures.'",[['NOPATH']],S3082464864
1421,9,middle_paths_unrestricted_16,"The pasta was mediocre, not as memorable as the pizza.",pasta,pizza,NONE,WORSE,0.600000,lex (unique),Samsung Fascinate vs Motorola DROID X,[['X/PROPN/compound/> fascinate/PROPN/ROOT/^ vs/ADP/prep/< Y/PROPN/pobj/<']],N2130818340
1422,692,middle_paths_unrestricted_16,-This maybe works too in the ps2 version but it will be harder than on the psp version.,ps2,psp,BETTER,WORSE,1.000000,lex (unique),"This is just like ""which is better, Ruby or PHP?"" or jazz vs. pop.","[['X/PROPN/ROOT/^ Y/PROPN/conj/<'], ['X/PROPN/ROOT/^ or/CONJ/cc/V Y/PROPN/conj/<']]",R2839423297
1425,1720,middle_paths_unrestricted_16,"Scala is not a better Java, it's an ML for JVM",Scala,Java,BETTER,WORSE,1.000000,lex (unique),Downloads: Pokemon Ruby Java Mobile Game,[['X/PROPN/compound/> Y/PROPN/ROOT/^']],E606781226
1426,127,middle_paths_unrestricted_16,"JUnit is, in turn, a Java version of Kent's Smalltalk testing framework.",Java,Smalltalk,BETTER,NONE,1.000000,lex (unique),"Sadly, at that time AMD wasn't 100% compatible to Intel; but still way"", 'better than that IBM/Cyrix shit.","[['X/PROPN/ROOT/^ Y/PROPN/pobj/<'], ['X/PROPN/ROOT/^ good/ADJ/conj/V Y/PROPN/pobj/<'], ['X/PROPN/ROOT/^ that/DET/det/V Y/PROPN/pobj/<']]",N2125709781


In [8]:
# all errors made by both classifiers
se_multi = se[~se.sentence.isin(se_unique.sentence.values.tolist())].copy()
lex_multi = lex[~lex.sentence.isin(lex_unique.sentence.values.tolist())].copy()
se_multi['origin'] = 'both'
lex_multi['origin'] = 'both'
assert sorted(se_multi.id.values.tolist()) == sorted(lex_multi.id.values.tolist())
shared = se_multi.copy()
shared['caption'] = 'shared'
shared['pred_if'] = shared.apply(lambda x: se_multi[se_multi.sentence == x['sentence']].predicted.values.tolist()[0],axis=1)
shared['pred_lex'] = shared.apply(lambda x: lex_multi[lex_multi.sentence == x['sentence']].predicted.values.tolist()[0],axis=1)
shared = shared.set_index('id')
shared = shared.join(conf,rsuffix="_r")[['sentence','object_a','object_b','pred_if', 'pred_lex', 'gold', 'most_frequent_percentage']]

In [9]:
print('Shared {}'.format(len(shared)))
print('SE Unique {} | Lex Unique {}'.format(len(se_unique),len(lex_unique)))
print('SE {} | Lex {}'.format(len(se),len(lex)))
assert len(shared) + len(se_unique) == len(se)
assert len(shared) + len(lex_unique) == len(lex)

Shared 607
SE Unique 220 | Lex Unique 484
SE 827 | Lex 1091


In [10]:
def class_confusion(df):
    d2 = defaultdict(int)
    d = defaultdict(int)
    for i, row in df.iterrows():
        k = 'Pred: {} - Gold: {}'.format(row['predicted'],row['gold'])
        d[k] += 1
    print('\n')
    pprint(sorted(d.items(),key=lambda x: x[1],reverse=True))

In [11]:
def analyze_errors(df,conf=True):
    df['s_len'] = df.apply(lambda x: len(x['sentence']),axis=1)
    print('### Confidence')
    print(df.most_frequent_percentage.value_counts())
    if conf:
        print('\n### Confusion')
        class_confusion(df)
    print('\n### Average sentence length')
    print(df.s_len.describe())
    print(df.groupby('most_frequent_percentage').mean().sort_values('s_len',ascending=False))
    

## Sentence Embedding Errors

In [12]:
analyze_errors(se_unique)
se_unique.to_csv('3_se.csv')

### Confidence
1.000000    156
0.800000     30
0.600000     26
0.400000      4
0.666667      1
0.833333      1
0.888889      1
0.500000      1
Name: most_frequent_percentage, dtype: int64

### Confusion


[('Pred: BETTER - Gold: NONE', 95),
 ('Pred: NONE - Gold: BETTER', 50),
 ('Pred: WORSE - Gold: NONE', 41),
 ('Pred: NONE - Gold: WORSE', 14),
 ('Pred: WORSE - Gold: BETTER', 12),
 ('Pred: BETTER - Gold: WORSE', 8)]

### Average sentence length
count    220.000000
mean     133.777273
std       65.082388
min       27.000000
25%       84.000000
50%      124.500000
75%      169.000000
max      373.000000
Name: s_len, dtype: float64
                                   id       s_len
most_frequent_percentage                         
0.833333                  1905.000000  255.000000
0.888889                  2310.000000  254.000000
0.666667                  2241.000000  161.000000
0.800000                  1493.833333  135.000000
1.000000                  1114.256410  132.980769
0.600000     

In [13]:
se_unique.sort_values('most_frequent_percentage',ascending=False).drop(columns=['caption','origin','id'])

,sentence,object_a,object_b,predicted,gold,most_frequent_percentage,s_len
3744,harvard Head Coach Tim Murphy opening statement: Obviously it as a very solid win over princeton last week.,harvard,princeton,BETTER,NONE,1.000000,107
3829,I think tapioca and white rice and potato flour are far better to make bread with.,potato,bread,BETTER,NONE,1.000000,82
3833,Remember before Nokia it was Motorola....,Nokia,Motorola,BETTER,NONE,1.000000,41
3835,rifle primers are a bit harder than pistol primers.,rifle,pistol,WORSE,BETTER,1.000000,51
3845,"It provides d rastic JSON parsing and serialization performance improvements "" The Rick (Hightower)/ Andrey duo spent a fair amount of time optimizing our JSON support, making Groovy 2.3's JSON support usually faster than all the JSON libraries available in the Java ecosystem.",Groovy,Java,BETTER,NONE,1.000000,277
3849,"Is Windows 8 better than Windows 7, its predecessor?",Windows 8,Windows 7,BETTER,NONE,1.000000,52
3851,"Critical bugs in the compiler and in the execution environment and without real intentions of improving it (I understand, they have their propietary solutions, like CUDA, which are better supported) To put something in production that uses OpenCL, the code must be flooded of workarounds.",CUDA,OpenCL,BETTER,NONE,1.000000,288
3855,"Otherwise, Microsoft will build hardware that better integrates with the operating software, and companies like Dell will suffer.",Microsoft,Dell,NONE,BETTER,1.000000,129
3859,"In his spare time, Lance is a mediocre softball player, a better soccer midfielder and a great little league baseball and basketball coach to son, Blaze.",softball,baseball,BETTER,NONE,1.000000,153
3873,"The Wildcats have beaten 2 solid Pac-10 teams in Stanford and oregon State, and also own a better than it seems win over Central michigan.",oregon,michigan,BETTER,NONE,1.000000,138


## LexNet Errors

In [14]:
analyze_errors(lex_unique)
lex_unique.to_csv('3_lex.csv')

### Confidence
1.000000    356
0.800000     66
0.600000     41
0.400000     15
0.833333      3
0.666667      1
0.714286      1
0.500000      1
Name: most_frequent_percentage, dtype: int64

### Confusion


[('Pred: NONE - Gold: BETTER', 195),
 ('Pred: BETTER - Gold: NONE', 117),
 ('Pred: BETTER - Gold: WORSE', 64),
 ('Pred: NONE - Gold: WORSE', 50),
 ('Pred: WORSE - Gold: NONE', 33),
 ('Pred: WORSE - Gold: BETTER', 25)]

### Average sentence length
count    484.000000
mean     115.241736
std       67.111704
min       17.000000
25%       70.000000
50%       97.000000
75%      141.000000
max      510.000000
Name: s_len, dtype: float64
                                   id       s_len
most_frequent_percentage                         
0.666667                   774.000000  154.000000
0.800000                  1115.318182  122.045455
1.000000                  1178.303371  115.808989
0.400000                  1582.266667  109.866667
0.600000                  1141.658537  106.268293
0.833333  

In [15]:
temp = lex_unique.sort_values('most_frequent_percentage',ascending=False).drop(columns=['caption','origin','sentencer','s_len'])
np = temp[temp.paths.str.startswith("[['NOP")]
print('Sentences without path: {}'.format(len(np)))

Sentences without path: 36


In [16]:
lex_unique

,id,caption,sentence,object_a,object_b,predicted,gold,most_frequent_percentage,origin,sentencer,paths,idr,s_len
1410,134,middle_paths_unrestricted_16,From (1) and (2) follows this question: Is it possible at all to write a web framework in Ruby that's faster than PHP?,Ruby,PHP,NONE,BETTER,1.000000,lex (unique),"Like michigan, pennsylvania is within a point of moving into the safer', 'Obama lean category.",[['X/PROPN/ROOT/^ Y/PROPN/conj/<']],K1630595994,118
1411,830,middle_paths_unrestricted_16,"Even without the heavy functional stuff, just as a basic OO language, Scala is far better than C++/Java/C#.",Scala,Java,NONE,BETTER,1.000000,lex (unique),"Obama did better in North carolina than Hillary had done in', 'pennsylvania and Indiana was considered too close to call until well', 'past midnight.","[['X/PROPN/ROOT/^ Y/PROPN/pobj/<'], ['X/PROPN/ROOT/^ do/VERB/advcl/V Y/PROPN/pobj/<']]",L1858700103,107
1412,378,middle_paths_unrestricted_16,"Again, this is a decent beer but its not my cup of tea.",beer,tea,WORSE,NONE,1.000000,lex (unique),"Why can't Ford puts better handling into the FWD Fusion if Honda can"", 'do it 10 years ago?",[['X/PROPN/nsubj/> put/VERB/ROOT/^ if/ADP/prep/< Y/PROPN/pobj/<']],T3173142468,55
1413,1618,middle_paths_unrestricted_16,"The sausage is decent too, three kinds sold by the sandwich ($7.50) or by the pound ($15.75).",sausage,sandwich,BETTER,NONE,1.000000,lex (unique),"georgia looked much better against missouri, particularly in the', 'second half than it did at any point against Buffalo and might be', 'putting things together now.","[['X/PROPN/nsubj/> look/VERB/ROOT/^ better/ADV/acomp/< against/ADP/prep/<', 'Y/PROPN/pobj/<']]",F804953248,93
1416,504,middle_paths_unrestricted_16,Java apps on Mac are slower and bulkier than their native Objective-C counterparts.,Java,Objective-C,BETTER,WORSE,1.000000,lex (unique),"With the wii remote (and later the shameless copy, the playstation', 'Move), faster precision was never this simple!","[['X/PROPN/ROOT/^ Y/PROPN/dobj/<'], ['X/PROPN/ROOT/^ remote/ADJ/nmod/V Y/PROPN/dobj/<'], ['X/PROPN/ROOT/^ the/DET/det/V Y/PROPN/dobj/<']]",T3203049748,83
1417,13,middle_paths_unrestricted_16,"As good as michigan has been defensively this season, virginia has been even better.",michigan,virginia,NONE,WORSE,0.800000,lex (unique),"We say 'Java is the gateway to hell.' Lisp is just a better"", 'alternative to the intelligent Internet: It runs on 14 different', ""platforms and doesn't need any type of virtual machine because it"", 'compiles directly to native machine instructions on all popular', ""microprocessor architectures.'",[['NOPATH']],S3082464864,84
1421,9,middle_paths_unrestricted_16,"The pasta was mediocre, not as memorable as the pizza.",pasta,pizza,NONE,WORSE,0.600000,lex (unique),Samsung Fascinate vs Motorola DROID X,[['X/PROPN/compound/> fascinate/PROPN/ROOT/^ vs/ADP/prep/< Y/PROPN/pobj/<']],N2130818340,54
1422,692,middle_paths_unrestricted_16,-This maybe works too in the ps2 version but it will be harder than on the psp version.,ps2,psp,BETTER,WORSE,1.000000,lex (unique),"This is just like ""which is better, Ruby or PHP?"" or jazz vs. pop.","[['X/PROPN/ROOT/^ Y/PROPN/conj/<'], ['X/PROPN/ROOT/^ or/CONJ/cc/V Y/PROPN/conj/<']]",R2839423297,87
1425,1720,middle_paths_unrestricted_16,"Scala is not a better Java, it's an ML for JVM",Scala,Java,BETTER,WORSE,1.000000,lex (unique),Downloads: Pokemon Ruby Java Mobile Game,[['X/PROPN/compound/> Y/PROPN/ROOT/^']],E606781226,46
1426,127,middle_paths_unrestricted_16,"JUnit is, in turn, a Java version of Kent's Smalltalk testing framework.",Java,Smalltalk,BETTER,NONE,1.000000,lex (unique),"Sadly, at that time AMD wasn't 100% compatible to Intel; but still way"", 'better than that IBM/Cyrix shit.","[['X/PROPN/ROOT/^ Y/PROPN/pobj/<'], ['X/PROPN/ROOT/^ good/ADJ/conj/V Y/PROPN/pobj/<'], ['X/PROPN/ROOT/^ that/DET/det/V Y/PROPN/pobj/<']]",N2125709781,72


## Shared

In [17]:
analyze_errors(shared,conf=False)
shared.to_csv('3_shared.csv')

### Confidence
1.000000    425
0.800000    101
0.600000     58
0.400000     15
0.833333      5
0.666667      3
Name: most_frequent_percentage, dtype: int64

### Average sentence length
count    607.000000
mean     122.565074
std       67.656519
min       16.000000
25%       78.000000
50%      105.000000
75%      152.000000
max      436.000000
Name: s_len, dtype: float64
                               s_len
most_frequent_percentage            
0.666667                  178.333333
0.800000                  130.584158
0.600000                  125.586207
0.833333                  120.800000
1.000000                  120.609412
0.400000                  101.733333


In [18]:
shared[shared.sentence.str.startswith('Python')]

,sentence,object_a,object_b,pred_if,pred_lex,gold,most_frequent_percentage,s_len
17,"Python , the best language a business can use without risking the lack of programmers ( Common Lisp is better, but I don't know anyone who knows it except for me)",Python,Lisp,NONE,NONE,WORSE,1.000000,162
550,"Python skills a plus, but not required, especially if you're solid at JavaScript.",Python,JavaScript,NONE,NONE,BETTER,0.600000,81
1612,"Python grew out of the need for a ""better"" Perl.",Python,Perl,WORSE,NONE,BETTER,1.000000,48
2120,"Python is nice and easy to use and learn but it runs like treacle much much slower than C, C++ or Java etc... if you're looking to explore the world of non C style programming try a functional language like Haskell it's a recursive programming language, no for loops and no variables it's quite interesting though.",Python,Haskell,BETTER,BETTER,WORSE,0.666667,314


## All

In [19]:
analyze_errors(pd.concat([shared,lex_unique,se_unique]))

### Confidence
1.000000    937
0.800000    197
0.600000    125
0.400000     34
0.833333      9
0.666667      5
0.500000      2
0.714286      1
0.888889      1
Name: most_frequent_percentage, dtype: int64

### Confusion


[('Pred: nan - Gold: WORSE', 292),
 ('Pred: NONE - Gold: BETTER', 245),
 ('Pred: nan - Gold: BETTER', 224),
 ('Pred: BETTER - Gold: NONE', 212),
 ('Pred: nan - Gold: NONE', 91),
 ('Pred: WORSE - Gold: NONE', 74),
 ('Pred: BETTER - Gold: WORSE', 72),
 ('Pred: NONE - Gold: WORSE', 64),
 ('Pred: WORSE - Gold: BETTER', 37)]

### Average sentence length
count    1311.000000
mean      121.742944
std        67.279072
min        16.000000
25%        76.000000
50%       105.000000
75%       153.000000
max       510.000000
Name: s_len, dtype: float64
                                   id       s_len
most_frequent_percentage                         
0.888889                  2310.000000  254.000000
0.666667                  1507.500000  170.000000
0.800000                  1233.6

## Misc

In [20]:
different = []
for i, row in shared.iterrows():
    se_s = se_multi[se_multi.sentence == row['sentence']].predicted.values.tolist()[0]
    lex_s = lex_multi[lex_multi.sentence == row['sentence']].predicted.values.tolist()[0]
    if se_s != lex_s:
        different.append(row['sentence'])
print('Different mistakes on same sentence: {}'.format(len(different)))


Different mistakes on same sentence: 130
